In [31]:
#importing libraries
import numpy as np
import pandas as pd
import functools as ft

In [32]:
#reading the csv file
df= pd.read_csv("E:/nyc-crime/NYPD_Complaint_Data_Historic(1).csv")

C:\Users\rajaa\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (18,20) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [33]:
#dropping the irrelevant columns
toDrop= ['CMPLNT_NUM','CMPLNT_FR_DT','CMPLNT_FR_TM','CMPLNT_TO_DT','CMPLNT_TO_TM','ADDR_PCT_CD',
'KY_CD','PD_CD','PD_DESC','CRM_ATPT_CPTD_CD','LAW_CAT_CD',
'LOC_OF_OCCUR_DESC','PREM_TYP_DESC','JURIS_DESC','JURISDICTION_CODE','PARKS_NM','HADEVELOPT',
'HOUSING_PSA','X_COORD_CD','Y_COORD_CD','SUSP_AGE_GROUP','SUSP_RACE','SUSP_SEX','TRANSIT_DISTRICT',
'Latitude','Longitude','Lat_Lon','PATROL_BORO','STATION_NAME','VIC_AGE_GROUP','VIC_RACE','VIC_SEX']
df.drop(toDrop, inplace=True, axis= 1)

In [34]:
#creating a column named year reffering to year of occurence
df['year']=df.RPT_DT.str.slice(-4)    #will consider the last 4 characters of the RPT_DT

In [35]:
#updating the values of the cells of RPT_DT referring to Month only
df['RPT_DT']= df.RPT_DT.str.slice(0,2)

#if there is month sliced as '5/' then / will be replaced with empty string
df['RPT_DT']=df.RPT_DT.str.replace("/","")

In [36]:
#changing the data typy of RPT_DT and year from object to Year
newType= {'RPT_DT': int, 'year': int}
df=df.astype(newType)

In [37]:
#creating a list containg the unique numbers of the year and month mentioned
years= pd.unique(df['year'])
months= pd.unique(df['RPT_DT'])

years.sort()
months.sort()

def getMultiIndex(yr,mon):
    l1= [yr[0]]*np.alen(mon)
    for i in yr:
        if i==yr[0]:
            continue
        l2= [i]*np.alen(mon)
        l1.extend(l2)
    m1=list(mon)
    for j in range(len(yr)-1):
        m1.extend(mon)

    arrays= [l1,m1]
    tuples= list(zip(*arrays))
    ind= pd.MultiIndex.from_tuples(tuples,names=['year','month'])
    return ind

index= getMultiIndex(years,months)

In [38]:
#renaming the column RPT_DT to Month and year to Year respectively
df= df.rename(columns= {'RPT_DT': 'Month', 'year': 'Year', 'OFNS_DESC': 'Category'})

In [39]:
#to retrieve data frame for all years on monthly basis crime frequency
def getYearDataFrame(NoOfYears):
    yearList= list(np.arange(NoOfYears))
    for i in np.arange(NoOfYears):
        yearList[i]= yearDataFrame(years[i])
    return yearList

#to retrieve data frame for monthly basis crime freqeuncy for a single year on the basis of the given year
def yearDataFrame(yearNumber):
    yearTemp= df.loc[df['Year']==yearNumber]
    monthFrame= getMonthDataFrame(yearTemp,len(months),yearNumber)
    yearDf= ft.reduce(lambda left, right: pd.merge(left,right,on='Category',how='outer'),monthFrame)
    yearDf=yearDf.fillna(0)
    return yearDf

#to retrive the monthly basis crime frequency for all months of a year
def getMonthDataFrame(yearTem,noOfMonths,yearNum):
    monthList= list(np.arange(noOfMonths))
    for i in np.arange(noOfMonths):
        monthList[i]= monthDataFrame(yearTem,months[i],yearNum)
    return monthList

#to retrieve the dataframe for crime frequency of a single month
def monthDataFrame(yearTempo,monthNumber,yearNo):
    monthTemp= yearTempo.loc[df['Month']==monthNumber]
    monthlyTempDf= monthTemp['Category'].value_counts().rename_axis('Category').reset_index(name="Month-"+str(monthNumber)+"("+str(yearNo)+")")
    return monthlyTempDf    

final_list= getYearDataFrame(len(years))

In [40]:
#merging all the year based dataframe into a single dataframe taking the Category as the common field
df_final= ft.reduce(lambda left, right: pd.merge(left,right,on='Category',how='outer'),final_list)

#replace all na values with 0
df_final= df_final.fillna(0)

In [41]:
#adding row showing total crime in each month
crimeInMonth= np.arange((len(months)*len(years))+1)
count=0
for i in np.arange(len(years)):
    for j in np.arange(len(months)):
               #crimeInMonth[count+1]= np.sum(df_final['Month-12(2015)'])
                crimeInMonth[count+1]= np.sum(df_final['Month-'+str(months[j])+"("+str(years[i])+')'])
                count+=1
               # crimeInMonth[count+1]= np.sum(df_final['Month-'+str(months[j])+"("+str(years[i])+")"])
    

crimeMonth= pd.DataFrame([crimeInMonth],columns=df_final.columns)
crimeMonth['Category']='TOTAL'

df_final= pd.concat([df_final,crimeMonth],ignore_index=True)

In [42]:
#retrieving all unique categories as a list
cat= df_final['Category']

In [43]:
#removing the Category column
df_final.drop(['Category'],inplace= True, axis=1)

In [44]:
#retrieving all teh data from the dataFrame df_final
data= df_final.values

In [45]:
#using the retrieved data from df_final as input to a new
#dataframe subDf taking the multiindex.
subDf= pd.DataFrame(data= data, columns= index)


In [46]:
#inserting the Category column in the dataframe subDf
subDf.insert(0,'Category',cat)
#selecting the Category column as the index
subDf= subDf.set_index('Category')

In [ ]:
#to Download the mutated dataframe as csv
subDf.to_csv('nycFrequency.csv')

In [48]:
# a function to calculate percentage of each crime in each month per year
def getPercentDataFrame(df1):
    dfTest= df1
    monTotal= dfTest.last_valid_index()
    for i in years:
        for j in months:
            if(dfTest[i][j][monTotal]>0):
                dfTest[i][j]*=(100/dfTest[i][j][monTotal])
                continue
            dfTest[i][j]=0
    return dfTest
#function call
subDf= getPercentDataFrame(subDf)

<ipython-input-48-82a759c921e7>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfTest[i][j]*=(100/dfTest[i][j][monTotal])


In [ ]:
#to download the mutated dataframe as csv
subDf.to_csv("nycPercentage.csv")